In [5]:
from sklearn.metrics import adjusted_rand_score
import re
from sklearn.metrics import normalized_mutual_info_score, accuracy_score
import os 
import pandas as pd

purity = 0.7
coverage = 70
# 5000,6,2,0.7,100
# 5000,8,2,0.7,70
general_folder = f"./results/p_{str(purity).replace('.', '')}_cov_{coverage}/D_2/"
directory = general_folder + 'csv/'
output_directory = general_folder + 'nmi/'

os.makedirs(output_directory, exist_ok=True)

# Regular expression pattern to match the file names and extract N, K, D, and df values
pattern = re.compile(r'N_(\d+)_K_(\d+)_D_(\d+)_df_(\d+)\.csv')

# Dictionary to store nmi values for each (N, K, D) combination
nmi_dict = {}

# Iterate through files in the specified directory
for filename in os.listdir(directory):
    # print('HERE')
    # Check if the file matches the pattern
    match = pattern.match(filename)
    if match:
        # Extract N, K, D, and df values from the file name
        N, K, D, df = map(int, match.groups())
        if N == 5000 and K == 8 and D == 2:
            # print('HERE')
            # Construct the full file path
            file_path = os.path.join(directory, filename)
            # Read the CSV file
            df_data = pd.read_csv(file_path)
            # Extract the specified columns if they exist
            if 'True_cluster' in df_data.columns and 'Pred_cluster' in df_data.columns:
                true_labels = df_data['True_cluster'].tolist()
                pred_labels = df_data['Pred_cluster'].tolist()
                # Calculate the Adjusted Rand Index (nmi)
                nmi = adjusted_rand_score(true_labels, pred_labels)
                # Add the nmi to the list for this combination of N, K, D
                if (N, K, D) not in nmi_dict:
                    nmi_dict[(N, K, D)] = []
                nmi_dict[(N, K, D)].append((df, nmi))

# Write the nmi lists to text files for each (N, K, D) combination
for (N, K, D), nmi_list in nmi_dict.items():
    # Sort the nmi values by df to ensure the order is correct (0 to 20)
    nmi_list_sorted = [nmi for _, nmi in sorted(nmi_list)]
    # Construct the output file path
    output_file_path = os.path.join(output_directory, f'nmi_N_{N}_K_{K}_D_{D}.txt')
    # Write the list of 20 nmi values to the text file
    with open(output_file_path, 'w') as f:
        f.write('\n'.join(map(str, nmi_list_sorted)))

    print(f'nmi values for N={N}, K={K}, D={D} written to {output_file_path}')


nmi values for N=5000, K=8, D=2 written to ./results/p_07_cov_70/D_2/nmi/nmi_N_5000_K_8_D_2.txt
